# スキャン画像の読み取り，位置合わせ

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

%config InlineBackend.figure_formats = {'png', 'retina'}

### 画像の読み込み

In [1]:
import glob
import os


def read_image(path):
    return cv2.imread(path, cv2.IMREAD_GRAYSCALE)


def read_images(dirname, ext=None):
    """
    Parameters
    ----------
    dirname: name of the directory.
    ext: file's extension such as ".png", ".jpg",...
    """
    if ext:
        pathr = os.path.join(dirname, "**", "*"+ext)
    else:
        pathr = os.path.join(dirname, "**")
    paths = glob.iglob(pathr, recursive=True)
    for p in paths:
        img = read_image(p)
        if img is None:
            continue
        yield p, img
    

### マーカーの位置検出

In [2]:
def find_markers(img, template, mask=None):
    if mask is not None:
        img = img * mask

    # テンプレートマッチング
    res = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)
    
    sort_idx = np.argsort(res)[-3:]
    m_res = res[sort_idx]
    # m_res が何を返すかわからないのでこの先はやってみないとわからない
    # http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_imgproc/py_template_matching/py_template_matching.html



### アフィン変換

In [3]:
def align_image(base_markers, img_markers, img):
    h, w = img.shape
    M = cv2.getAffineTransform(img_markers, base_markers)
    new_img = cv2.warpAffine(img, M, (h, w), borderValue=255)  # 境界の外は白で埋める
    return new_img

### 画像の保存

In [4]:
def save_image(path, img):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    cv2.imwrite(path, img)

### 実行

In [0]:
TEMP_PATH = ""
BASE_PATH = ""
DIRNAME = ""
SAVE_DIRNAME = ""

template = read_image(TEMP_PATH)
baseimg = read_image(BASE_PATH)
base_markers = find_markers(baseimg, template)
images = read_images(DIRNAME)
for p, img in images:
    markers = find_markers(img, templage)
    new_img = align_image(base_markers, markers, img)

    plt.imshow(new_img, cmap="gray")
    plt.show()

    # q = p.replace(DIRNAME, SAVE_DIRNAME)
    # save_image(q, new_img)